**Imports**

In [15]:
import httpx
import shutil
import asyncio
import pandas as pd

from os import path, mkdir
from exchanges import Binance, Coinalyze
from models import Ticker, Timeframe

**Utility**

In [16]:
def highest_volume(tickers: list[Ticker]) -> list[Ticker]:
    """Sorts tickers based on volume and removes tickers with no volume."""
    filtered = filter(lambda x: x.price > 0 and x.volume > 0, tickers)
    tickers = sorted(filtered, key=lambda x: x.price * x.volume, reverse=True)
    return tickers

def filter_symbols(market: str, tickers: list[Ticker], blacklist: list[str] = []) -> list[Ticker]:
    """Returns list of tickers denominated in the provided market, excluding symbols that are blacklisted."""
    filtered = filter(lambda x: x.symbol.endswith(market) and not blacklisted(x.symbol, blacklist), tickers)
    return list(filtered)

def blacklisted(symbol: str, blacklist: list[str]) -> bool:
    """Checks if a blacklisted symbol is part of a given market and returns True if it is."""
    if len(blacklist) > 0:
        for item in blacklist:
            if symbol.startswith(item) or symbol.endswith(item):
                return True
    return False

**Data Selection**

In [17]:
binance = Binance()
markets = binance.markets()

blacklist = ["UPUSDT", "DOWNUSDT", "BEARUSDT", "BULLUSDT"]
currencies = ["EUR", "JPY", "GBP", "CAD", "CNY", "CHF", "AUD"]
stablecoins = ["TUSD", "BUSD", "USDC", "PAX", "USDP", "DAI", "GUSD", "USDD", "USTC", "UST", "USDS", "FDUSD"]

usdt_markets = filter_symbols("USDT", markets, blacklist + currencies + stablecoins)
usdt_volume = highest_volume(usdt_markets)

btc_markets = filter_symbols("BTC", markets)
btc_volume = highest_volume(btc_markets)

binance_markets = ["BTCUSDT", "ETHUSDT"]
coinalyze_markets = ["BTCUSD_PERP.A", "ETHUSD_PERP.A"]

**Data Download**

In [18]:
usdt_dl = usdt_volume[:100]

print(f"Downloading {len(usdt_dl)} markets...")
usdt_data = []
async with httpx.AsyncClient() as client:
    usdt_symbols = [ticker.symbol for ticker in usdt_dl]
    for symbol in usdt_symbols:
        usdt_data.append(binance.kline(client, symbol, Timeframe.MINUTLY, 1000))
    usdt_data = await asyncio.gather(*usdt_data)
    usdt_data = dict(zip(usdt_symbols, usdt_data))
print("Finished downloading.")

Finished downloading.


In [19]:
try:
    coinalyze = Coinalyze()
    spot = coinalyze.spot_markets()
    futures = coinalyze.futures_markets()

    oi = coinalyze.open_interest(coinalyze_markets)
    fu = coinalyze.funding(coinalyze_markets)
except KeyError as err:
    print(err)

**Data Persistence**

Execute to save the downloaded kline data into individual csv-files for each downloaded market.

In [20]:
data_path = path.join("..", "data")

if path.exists(data_path):
    shutil.rmtree(data_path)

mkdir(data_path)

symbols = {
    "USDT": usdt_dl,
    "BTC": btc_volume
}

for symbol, data in symbols.items():
    df = pd.DataFrame.from_dict(data)
    df.to_csv(path.join(data_path, f"{symbol}_markets.csv"), index=False)

for symbol, kline in usdt_data.items():
    df = pd.DataFrame.from_dict(kline).set_index("time")
    df.to_csv(path.join(data_path, f"{symbol}.csv"))

# Remove data from memory
df = None
usdt_data = None